# 尋找均線交易的最優組合。
抓出使用均線的最佳買入點、最佳賣出點組合。
此處，先固定買入點、賣出點的關係相同。讓運算時間降低。

In [2]:
import pandas, os, re, numpy
import talib

# 路徑函數
def os_path(list) : 
    os_path = os.path.dirname(os.path.dirname(os.getcwd()))
    for i in list : 
        os_path = os.path.join(os_path, i)
    return os_path

# 回測函數一、買賣邏輯同。
def return_test_logic_same(df, indicator_1, indicator_2) : # indicator_1、indicator_2，都是字串。
    # 建立一個儲存買賣資訊的 list，內部每個元素都用 dict 儲存。格式 : {"買入價格" : buy_value ,"賣出價格" : sale_value}
    Order_list = []
    sale_index = -1
    for index, column_data in df.iterrows() : # 鄵，少一個存放目前是否有買入資料的 list 幹妳量，難怪會有一這麼多交易次數。
        if index > sale_index : 
            pass
        else : 
            continue
        
        # 計算買入點、售出點。
        try :  # 5_MA在前四個欄位，是遺漏值，此時無法比較大小，會報錯。 
            if column_data[indicator_1] > column_data[indicator_2] : 
                buy_value = column_data["stock_price"]
            else : 
                continue
            
            # 再次遍歷整個 data，抓出賣出點。
            for index_2, column_data_2 in df.iloc[index+1:, :].iterrows() :
                if column_data_2[indicator_1] < column_data_2[indicator_2] : 
                    sale_value = column_data_2["stock_price"]
                    sale_index = index_2 # 避免重複交易，透過記住賣出點的 index，讓下一輪的買入點，必定要在這個賣出點後面。
                    # 儲存買入、賣出資訊。
                    Order_list.append({"real_profitability" : 100*(sale_value - buy_value)/buy_value - (0.1425*0.65 + 0.3)})
                    break
                else : 
                    continue

        except : 
            pass

    # 買賣資訊 dataframe化，用於後續串接。
    df_Order =  pandas.DataFrame(Order_list)

    # 扣除交易手續費，以 永豐金為例，0.1425%為交易手續費。交易稅，個股為 0.3%，當充為 0.15%。
    # 創建真實獲利率函數。
    result = {"real_profitability" : df_Order["real_profitability"].sum(), "order_sum" : len(df_Order) , "indicator_1" : indicator_1, "indicator_2" : indicator_2}
    return result

# 此函數，在上一個函數的基礎上，將賣出點進行修正，根據當前股價是否低於長期指標，試圖修正短期指標不夠敏感的問替。
def return_test_logic_different(df, indicator_1, indicator_2) : # indicator_1、indicator_2，都是字串。
    # 建立一個儲存買賣資訊的 list，內部每個元素都用 dict 儲存。格式 : {"買入價格" : buy_value ,"賣出價格" : sale_value}
    Order_list = []
    sale_index = -1
    for index, column_data in df.iterrows() : # 鄵，少一個存放目前是否有買入資料的 list 幹妳量，難怪會有一這麼多交易次數。
        if index > sale_index : 
            pass
        else : 
            continue
        
        # 計算買入點、售出點。
        try :  # 5_MA在前四個欄位，是遺漏值，此時無法比較大小，會報錯。 
            if column_data[indicator_1] > column_data[indicator_2] : 
                buy_value = column_data["stock_price"]
            else : 
                continue
            
            # 再次遍歷整個 data，抓出賣出點。
            for index_2, column_data_2 in df.iloc[index+1 : , :].iterrows() :
                if column_data_2["stock_price"] < column_data_2[indicator_2] : 
                    sale_value = column_data_2["stock_price"]
                    # 儲存買入、賣出資訊。
                    Order_list.append({"real_profitability" : 100*(sale_value - buy_value)/buy_value - (0.1425*0.65 + 0.3)})
                    break
                else : 
                    continue


            # 再次遍歷整個 data，抓出短期指標低於長期指標的座標，讓下一個迴圈根據此座標
            for index_3, column_data_3 in df.iloc[index+1 : , :].iterrows() :
                if column_data_3[indicator_1] < column_data_3[indicator_2] : 
                    sale_index = index_3
                    break
                else : 
                    continue
        except : 
            pass

    # 買賣資訊 dataframe化，用於後續串接。
    df_Order =  pandas.DataFrame(Order_list)

    # 扣除交易手續費，以 永豐金為例，0.1425%為交易手續費。交易稅，個股為 0.3%，當充為 0.15%。
    # 創建真實獲利率函數。
    result = {"real_profitability" : df_Order["real_profitability"].sum(), "order_sum" : len(df_Order) , "indicator_1" : indicator_1, "indicator_2" : indicator_2}
    return result

# 均線交易(買賣同)

In [25]:
df = pandas.read_csv(os_path(["database", "APPL_stock_2023.6.12_2023.7.13.csv"]))
result_list = []
error_list = []

# 引入 time 模組
import time

# 開始測量
start = time.time()
for i in range(2,21) : 
    for i2 in range(5, 61) : 
        if i > i2 : 
            continue
        else : 
            pass
        
        df_MA = df[["stock_price"]]
        df_MA[f"{i}"] = talib.SMA(df_MA.stock_price, timeperiod= i)
        df_MA[f"{i2}"] = talib.SMA(df_MA.stock_price, timeperiod= i2)
        try : 
            result = return_test_logic_same(df_MA, f"{i}", f"{i2}")
            result_list.append(result)
        except : 
            error = {"indicator_1" : f"{i}", "indicator_2" : f"{i2}"}
# 結束測量
end = time.time()
# 輸出結果
print("執行時間：%f 秒" % (end - start)) # 執行時間：87.132436 秒

df_result = pandas.DataFrame(result_list)
df_result.to_csv(os_path(["output", "最佳均線獲利解_買賣同.csv"]), index= False)



執行時間：71.841973 秒


In [26]:
# theard(多線呈)、核(身體)
# Flask

In [27]:
df_result[df_result["real_profitability"] == df_result["real_profitability"].max()]

,real_profitability,order_sum,indicator_1,indicator_2
506,1.305529,1,20,34


In [28]:
df_result[df_result["real_profitability"] == df_result["real_profitability"].min()]

,real_profitability,order_sum,indicator_1,indicator_2
0,-4.859891,11,2,5


# 修正均線交易。
此函數，在上一個函數的基礎上，將賣出點進行修正，根據當前股價是否低於長期指標，試圖修正短期指標不夠敏感的問替。

In [29]:
df = pandas.read_csv(os_path(["database", "APPL_stock_2023.6.12_2023.7.13.csv"]))
result_list = []
error_list = []

# 引入 time 模組
import time

# 開始測量
start = time.time()
for i in range(2,21) : 
    for i2 in range(5, 61) : 
        if i > i2 : 
            continue
        else : 
            pass
        
        df_MA = df[["stock_price"]]
        df_MA[f"{i}"] = talib.SMA(df_MA.stock_price, timeperiod= i)
        df_MA[f"{i2}"] = talib.SMA(df_MA.stock_price, timeperiod= i2)
        try : 
            result = return_test_logic_different(df_MA, f"{i}", f"{i2}")
            result_list.append(result)
        except : 
            error = {"indicator_1" : f"{i}", "indicator_2" : f"{i2}"}

# 結束測量
end = time.time()
# 輸出結果
print("執行時間：%f 秒" % (end - start)) # 執行時間：118.746526 秒

df_result = pandas.DataFrame(result_list)
df_result.to_csv(os_path(["output", "最佳均線獲利解_均線修正.csv"]), index= False)

執行時間：118.746526 秒


In [30]:
df_result[df_result["real_profitability"] == df_result["real_profitability"].max()]

,real_profitability,order_sum,indicator_1,indicator_2
169,20.389032,31,5,10


In [31]:
df_result[df_result["real_profitability"] == df_result["real_profitability"].min()]

,real_profitability,order_sum,indicator_1,indicator_2
138,-20.536417,20,4,33
192,-20.536417,20,5,33


In [32]:
len(df_result[df_result["real_profitability"] < 0])

881

In [33]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   real_profitability  909 non-null    float64
 1   order_sum           909 non-null    int64  
 2   indicator_1         909 non-null    object 
 3   indicator_2         909 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 28.5+ KB


# 目前預想的改進架構
首先，既然目前技術上還沒辦法採用多線性編成去加速，巢狀迴圈。不然改變視角，

我們在台股，971個上市證卷中，先選取能夠小額交易的證卷，將這些證卷根據不同技術指標的公定標準，去計算，找出最值得信賴的指標。

接著，將其中獲利較佳的證卷，選出，假設其為，採用某技術指標，較有潛力的證卷，基於該證卷，對某技術指標做優化，找出最優解，依照此最優解進行投資。

如 本次蘋果在 均線交易上，以 4_MA，對應 9_MA，有最優解。

不過嘛，我們該知道的事，這種最優解，可能是來自於超大量交易而來的結果，你可能在這個月的期間，交易了10多次，搞的心情很糟。

這在心理上的評估，也是必須去理解的。

其次，可能每個月的最優解都不相同，此時這個月是 4、9 下個月變成，5、8，誰也說不準，

這個時候也許應該要使用函數來 train 一波大的，預期最佳策略為何。

函數目標 : 
根據未來可以使用的預期變數(月份、上個月的最優解)，得到該月應該採用的最優解組合，


In [4]:
import pandas, os, re, numpy
import talib

df_修正 = pandas.read_csv(os_path(["output", "最佳均線獲利解_均線修正.csv"]))
df_買賣同 = pandas.read_csv(os_path(["output", "最佳均線獲利解_買賣同.csv"]))

print(df_修正["real_profitability"].mean())
print(df_買賣同["real_profitability"].mean())

-9.069512174080856
-1.1297088319411372
